In [8]:
import os
import pickle

In [2]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

In [3]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [4]:
import math

In [5]:
def preprocessing(words):
    mod_words = []
    m_words = []
    symbols = ['.',',',':','?','!','<','>','(',')','#','--','-','$','@','%','``',';',"''"]
    for word in words:
        m_words.append(ps.stem(word))
    for word in m_words:
        if word not in stopWords:
            if word.isalpha():
                if len(word) > 2 :
                    mod_words.append(word.lower())
    return mod_words
 

In [6]:
classes = []
mypath = '20_newsgroups'
folders = [f for f in os.listdir(mypath)]
for folder in folders:
    cls = []
    f_path = os.path.join(mypath,folder)
    for files in os.listdir(f_path):
        text = ''
        path = os.path.join(f_path,files)
        with open(path,'r',errors='ignore',encoding="utf8") as f:
            text += f.read()
        cls.append(text)
    classes.append(cls)


In [34]:
out = open('./pickle/task1/folders.pkl','wb')
pickle.dump(folders,out)
out.close()

In [9]:
out = open('./pickle/task1/classes.pkl','wb')
pickle.dump(classes,out)
out.close()

In [10]:
classes_train = []
classes_test = []
for i in range(20):
    train = classes[i][:850]
    test = classes[i][850:]
    classes_test.append(test)
    classes_train.append(train)

In [11]:
classes_train_text = []
for i in range(20):
    text = ''
    for j in range(len(classes_train[i])):
        text += classes_train[i][j] + ' '
    classes_train_text.append(text)

In [12]:
classes_train_words = []

for text in classes_train_text:
    classes_train_words.append(word_tokenize(text))


In [13]:
len(classes_train_words[19])

369219

In [14]:
mod_classes_train_words = []

for words in classes_train_words:
    mod_classes_train_words.append(preprocessing(words))


In [15]:
out = open('./pickle/task1/mod_classes_train_words.pkl','wb')
pickle.dump(mod_classes_train_words,out)
out.close()

In [16]:
classes_dict = []
for words in mod_classes_train_words:
    cls_dict = {}
    for word in words:
        if word in cls_dict:
            cls_dict[word] += 1
        else:
            cls_dict[word] = 1
    classes_dict.append(cls_dict)


In [17]:
out = open('./pickle/task1/classes_dict.pkl','wb')
pickle.dump(classes_dict,out)
out.close()

In [18]:
len(classes_dict)

20

In [19]:
vocab = set()
for dicti in classes_dict:
    for key in dicti.keys():
        vocab.add(key)

In [20]:
out = open('./pickle/task1/vocab.pkl','wb')
pickle.dump(vocab,out)
out.close()

In [21]:
Vocab_size = len(vocab)

In [22]:
def calc_freq(dicti,k,size,Vocab_size):
    for key in dicti.keys():
        dicti[key] += k
    size += k * Vocab_size
    
    freq = {}
    for key in dicti.keys():
        freq[key] = math.log10(dicti[key]/size)
    return freq
    

In [23]:
def probability(words,freq,k,size):
    p = 0
    smoothing = (k/(size+Vocab_size)) 
    for word in words :
        if word in freq:
            x = (freq[word])
            p += math.log10((10 ** x) + smoothing)
        else :
            p += math.log10(smoothing)
    return p

In [29]:
smooth = [1,5,10,100]
accuracy = []
all_clses_freq = []
for k in smooth:
    classes_freq = []
    for i in range(20):
        classes_freq.append(calc_freq(classes_dict[i],k,len(mod_classes_train_words[i]),Vocab_size))
   
    all_clses_freq.append(classes_freq)

    acc = []
    for i in range(20):
        count = 0
        for samp in classes_test[i]:
            samp_words = word_tokenize(samp)
            mod_samp_words = preprocessing(samp_words)
            classes_prob = []
            for j in range(20):
                classes_prob.append(probability(mod_samp_words,classes_freq[j],k,len(mod_classes_train_words[j])))
            max_pos =classes_prob.index(max(classes_prob))
            if max_pos == i:
                count += 1
        count /= len(classes_test[i])
        acc.append(count)
    print(acc)
    accuracy.append(acc)
        

[0.9, 0.8466666666666667, 0.7733333333333333, 0.58, 0.74, 0.8, 0.88, 0.9066666666666666, 0.8733333333333333, 0.92, 0.84, 0.9387755102040817, 0.76, 0.9866666666666667, 0.9, 0.8866666666666667, 0.94, 0.8266666666666667, 0.6066666666666667, 0.9666666666666667]
[0.9266666666666666, 0.86, 0.8066666666666666, 0.5333333333333333, 0.82, 0.8133333333333334, 0.9133333333333333, 0.9133333333333333, 0.94, 0.9266666666666666, 0.84, 0.9659863945578231, 0.8066666666666666, 0.9933333333333333, 0.92, 0.9133333333333333, 0.9466666666666667, 0.76, 0.6733333333333333, 0.9733333333333334]
[0.92, 0.88, 0.8333333333333334, 0.4866666666666667, 0.8333333333333334, 0.82, 0.9133333333333333, 0.92, 0.9466666666666667, 0.9266666666666666, 0.82, 0.9795918367346939, 0.82, 0.9866666666666667, 0.9333333333333333, 0.9, 0.9466666666666667, 0.7866666666666666, 0.7133333333333334, 0.9666666666666667]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]


In [31]:
all_clses_freq[0]

[{'xref': -2.664132320434794,
  'newsgroup': -2.2601128496621063,
  'path': -2.282568011342195,
  'wvhorn': -2.944611216235386,
  'william': -2.91635445667196,
  'vanhorn': -2.944611216235386,
  'subject': -2.252260561609826,
  'krillean': -2.8426216776980544,
  'photographi': -2.7935582724205594,
  'sender': -2.477332750668737,
  'news': -2.56455206504605,
  'organ': -2.248570696746661,
  'ohio': -2.928491220032232,
  'state': -2.6679756595165545,
  'univers': -2.473615514149786,
  'refer': -2.3550464405528655,
  'date': -2.2711953163717555,
  'tue': -2.8187610533074654,
  'apr': -2.2813776191435475,
  'gmt': -2.31988724027148,
  'line': -2.267725568366443,
  'articl': -2.382668448115406,
  'alex': -2.909568467328668,
  'alexand': -2.914647992857943,
  'zijdenbo': -2.9320220889273654,
  'write': -2.3762122648798503,
  'read': -2.6867109031311665,
  'post': -2.689744404278726,
  'kirlian': -2.8572951843059906,
  'whatev': -2.8694017377676078,
  'spell': -2.91635445667196,
  'could': -2

In [32]:
out = open('./pickle/task1/all_clses_freq.pkl','wb')
pickle.dump(all_clses_freq,out)
out.close()

In [33]:
out = open('./pickle/task1/accuracy.pkl','wb')
pickle.dump(accuracy,out)
out.close()

In [26]:
accuracy[0]

[0.9333333333333333,
 0.9066666666666666,
 0.8333333333333334,
 0.49333333333333335,
 0.8333333333333334,
 0.8333333333333334,
 0.9266666666666666,
 0.92,
 0.9466666666666667,
 0.92,
 0.7933333333333333,
 0.9931972789115646,
 0.8,
 0.9866666666666667,
 0.9466666666666667,
 0.9133333333333333,
 0.96,
 0.7666666666666667,
 0.6866666666666666,
 0.9866666666666667]

In [36]:
file = '20_newsgroups/rec.sport.baseball/100521'
with open(file,errors='ignore',encoding="utf8") as f:
    c = f.read() 

In [38]:
words = preprocessing(word_tokenize(c))
smooth_prob = []
j =0
for k in smooth :
    c_prob = []
    for i in range(20):
        c_prob.append(probability(words,all_clses_freq[j][i],k,len(mod_classes_train_words[j])))
    j += 1
    smooth_prob.append(c_prob)

In [39]:
#folders[smooth_prob[0].index(max(smooth_prob[0]))]

In [40]:
for k in range(len(smooth)):
    print('K is ',smooth[k])
    for i in range(20):
        print('Log Probability of class {} : {}'.format(folders[i],smooth_prob[k][i]))
    
    val = max(smooth_prob[k])
    fol = folders[smooth_prob[k].index(val)]
    print('Test case belongs to {} with log probability {}'.format(fol,val))
    print('############################')
    

K is  1
Log Probability of class sci.med : -486.31589762767226
Log Probability of class comp.sys.mac.hardware : -485.3024400694125
Log Probability of class alt.atheism : -486.15019510222146
Log Probability of class talk.religion.misc : -478.7915755521069
Log Probability of class comp.graphics : -496.2869964854297
Log Probability of class sci.electronics : -497.7571726439765
Log Probability of class comp.windows.x : -512.5236861735536
Log Probability of class talk.politics.guns : -474.94594094762584
Log Probability of class talk.politics.mideast : -500.8600600055008
Log Probability of class rec.autos : -490.42083615539985
Log Probability of class comp.os.ms-windows.misc : -503.01513900046956
Log Probability of class soc.religion.christian : -489.4137055536515
Log Probability of class comp.sys.ibm.pc.hardware : -507.81398847716133
Log Probability of class rec.motorcycles : -496.92033601306963
Log Probability of class sci.crypt : -495.81210229450323
Log Probability of class sci.space : -4